# Traffic Sign Detection and Classification
This work focuses on the detection of color and shapes regarding traffic signs under many different circumstances and combinations of illumination, angle, and contrast.

The first step is to import all the necessaries libraries that will be used in this project.

In [114]:
import cv2
import os

import xml.etree.ElementTree as ET
import imutils
import numpy as np
import math

### Global Variables
- `FILENAME`: Define the road or parse all roads inside the `data` folder. `ALL` to show all images.
- `TYPE`: `WINDOW` to open an window with the image, `FILE` to create a file with the file.  
- `DEBUG`: Used for debugging purposes.

In [115]:
FILENAME = "ALL"
OUTPUT = "WINDOW"
DEBUG = False

### Image Data

Class used to store information about the image.

In [116]:
class Data:
    def __init__(self, filename) -> None:
        self.filename = filename
        self.image = cv2.imread(filename)

### Matching

Alternative method where we try to detect traffic signs using SIFT descriptors and FLANN detector.

Only applied when no traffic sign of the same type of the one used as the template for matching has been found by our main strategy (colour and shape detection).

In [117]:
def draw_matches(image_data, dst, kp2, good, sign):
    image_data.image = cv2.polylines(image_data.image, [np.int32(dst)], True, 255, 3, cv2.LINE_AA)
    list_kp2 = [kp2[mat.trainIdx].pt for mat in good]
    x,y = sum(i for i, _ in list_kp2), sum(j for _, j in list_kp2)
    x_mean = int(x/len(good))
    y_mean = int(y/len(good))
    cv2.putText(image_data.image, sign, (x_mean - 35,y_mean - 10), cv2.FONT_HERSHEY_SIMPLEX,0.5, (255, 255, 255), 2)

def matching(image1, image2, sign):
    MIN_MATCH_COUNT = 5

    img1 = cv2.imread(image1,cv2.IMREAD_GRAYSCALE) # queryImage
    img2 = cv2.imread(image2,cv2.IMREAD_GRAYSCALE) # trainImage
    
    if sign == "red triangle":
        scale_percent = 50 # percent of original size
        width = int(img1.shape[1] * scale_percent / 100)
        height = int(img1.shape[0] * scale_percent / 100)
        dim = (width, height)

        # resize image
        img1 = cv2.resize(img1, dim, interpolation = cv2.INTER_AREA)

    img1 = cv2.GaussianBlur(img1, (5, 5), 0)
    img2 = cv2.GaussianBlur(img2, (5, 5), 0)

    # Initiate SIFT detector
    sift = cv2.SIFT_create()

    # find the keypoints and descriptors with SIFT
    kp1, des1 = sift.detectAndCompute(img1,None)
    kp2, des2 = sift.detectAndCompute(img2,None)

    FLANN_INDEX_KDTREE = 0
    index_params = dict(algorithm = FLANN_INDEX_KDTREE, trees = 5)
    search_params = dict(checks = 50)

    flann = cv2.FlannBasedMatcher(index_params, search_params)

    matches = flann.knnMatch(des1,des2,k=2)

    # store all the good matches as per Lowe's ratio test.
    good = []
    for m,n in matches:
        if m.distance < 0.7*n.distance:
            good.append(m)

    if len(good)>MIN_MATCH_COUNT:
        src_pts = np.float32([kp1[m.queryIdx].pt for m in good]).reshape(-1,1,2)
        dst_pts = np.float32([kp2[m.trainIdx].pt for m in good]).reshape(-1,1,2)

        M, _ = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC, 5.0)
        
        if M is None:
            return (None, None, None, None)
        
        h,w = img1.shape
        pts = np.float32([[0,0],[0,h-1],[w-1,h-1],[w-1,0]]).reshape(-1,1,2)
        dst = cv2.perspectiveTransform(pts,M)

    else:
        return (None, None, None, None)
    
    return (dst, kp2, good, sign)

### Shape detection class

This class is used to detect the shapes present in the image.

In [118]:
class ShapeDetector:
    def __init__(self, red, blue, image) -> None:
        self.red = red
        self.blue = blue
        self.image = image

    def _shape_name(self, contour, colour):
        peri = cv2.arcLength(contour, True)
        factor = 0.02 if colour == "blue" else 0.01
        approx = cv2.approxPolyDP(contour, factor * peri, True)

        if len(approx) == 3:
            return "triangle"
        
        elif len(approx) == 4:
            (_, _, width, height) = cv2.boundingRect(approx)
            aspect_ratio = width / float(height)

            # a square will have an aspect ratio that is approximately equal to one, otherwise, the shape is a rectangle
            return "square" if 0.90 <= aspect_ratio <= 1.10 else "rectangle"
        
        elif 6 <= len(approx) <= 7:
            # if the shape is a triangle, it can have 6 or 7 vertices (due to the corner curves): verify if the sizes of its sides are on a similar ratio of those in a triangle (three of them must be way longer)
            distances = []
            for x,y in zip(approx, approx[1:]):
                d = math.sqrt((x[0][1]-y[0][1])*(x[0][1]-y[0][1]) + (x[0][0]-y[0][0])*(x[0][0]-y[0][0]))
                distances.append(d)

            distances.sort()
            if distances[len(approx) - 5] < 1/4 * distances[len(approx) - 4] or distances[len(approx) - 4] < 1/4 * distances[len(approx) - 3]:
                return  "triangle"
        
        elif len(approx) == 8 and colour == "red":
            return  "stop"

        return "unidentified"

    def _shape_countours(self, image, colour):
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY) 
        blurred = cv2.GaussianBlur(gray, (7, 7), 0)
        blurred = cv2.threshold(blurred, 20, 255, cv2.THRESH_BINARY)[1]

        final_contours = []

        # find contours in the thresholded image and initialize the shape detector
        countours = cv2.findContours(blurred.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        countours = imutils.grab_contours(countours)

        # Draw circles
        radius = int(self.image.shape[0]*self.image.shape[1]*0.00005)
        circles = cv2.HoughCircles(blurred, cv2.HOUGH_GRADIENT_ALT, 1.5, 30, param1=250, param2=0.75, minRadius=radius)

        if circles is None: circles = [[]]
        circles = np.uint16(np.around(circles))

        processed_centers = {}
        for i in circles[0,:]:
            if (i[0],i[1]) in processed_centers.keys():
                if processed_centers[(i[0],i[1])] < i[2]:
                    processed_centers[(i[0],i[1])] = i[2]
            else:
                processed_centers[(i[0], i[1])] = i[2]

        for c in countours:
            # Minimum area of a valid contour
            AREA = 1000
            if cv2.contourArea(c) < AREA:
                continue
            
            # Shape of the Image
            shape = self._shape_name(c, colour)
            if shape == "unidentified":
                continue

            # Compute the center of the contour, then detect the name of the shape using only the contour
            M = cv2.moments(c)
            cX = int((M["m10"] / (M["m00"] + 1e-7))) #* ratio)
            cY = int((M["m01"] / (M["m00"] + 1e-7))) #* ratio)
            
            c = c.astype("float")
            c = c.astype("int")
            
            if not (shape == "stop" and colour == "blue"): 
                final_contours.append((shape, c, colour))

                # Draw and write shape name
                cv2.drawContours(image, [c], -1, (0, 255, 0), 2)
                classification = f"{colour} {shape}" if shape != "stop" else "stop sign"
                cv2.putText(image, classification, (cX - 35, cY - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)

        for (i[0],i[1]) in list(processed_centers.keys()):
            circle = {'center': (i[0],i[1]), 'radius': (processed_centers[i[0],i[1]])}
            final_contours.append(("circle", circle, colour))

            # Draw the circle, center and write the shape name
            cv2.circle(image,(i[0],i[1]),processed_centers[i[0],i[1]],(0,255,0),2)
            cv2.circle(image,(i[0],i[1]),2,(0,0,255),3)
            classification = f"{colour} circle"
            cv2.putText(image, classification, (i[0] - 35, i[1] - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)

        return final_contours

    def find_shape(self):
        red_contours = self._shape_countours(self.red, "red")
        blue_contours = self._shape_countours(self.blue, "blue")

        return red_contours + blue_contours


### Colour Detection Class

This class is used to detect the colours red and blue of the image.

In [119]:
class ColorDetector:
    def __init__(self, image) -> None:
        self.image = image

    def find_color(self):
        hsv = cv2.cvtColor(self.image, cv2.COLOR_BGR2HSV)

        h,s,v = cv2.split(hsv)
        hue = hsv[:, :, 0].mean()
        saturation = hsv[:, :, 1].mean()

        # CLAHE: used to increase contrast
        clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
        swithCLAHE = clahe.apply(s)
        vwithCLAHE = clahe.apply(v)

        hsv = cv2.merge([h, swithCLAHE, vwithCLAHE])

        # Generate lower mask and upper mask of red, depending on the average saturation and hue of the image
        if saturation < 61:
            if hue < 70:
                red_mask1 = cv2.inRange(hsv, (0,90,50), (20,255,255))
                red_mask2 = cv2.inRange(hsv, (160,90,50), (180,255,255))
            else:
                red_mask1 = cv2.inRange(hsv, (0,90,50), (10,255,255))
                red_mask2 = cv2.inRange(hsv, (170,90,50), (180,255,255))
        else:
            if hue < 70:
                red_mask1 = cv2.inRange(hsv, (0,30,30), (20,255,255))
                red_mask2 = cv2.inRange(hsv, (160,30,30), (180,255,255))
            else:
                red_mask1 = cv2.inRange(hsv, (0,30,30), (10,255,255))
                red_mask2 = cv2.inRange(hsv, (170,30,30), (180,255,255))

        # Merge the mask and crop the red regions
        red_mask = cv2.bitwise_or(red_mask1, red_mask2)

        # Generate mask (100-140) of blue, depending on the average saturation of the image
        if saturation < 40:
            blue_mask = cv2.inRange(hsv, (100,130,50), (140,255,255))
        else:
            blue_mask = cv2.inRange(hsv, (100,220,50), (140,255,255))

        red_mask = cv2.morphologyEx(red_mask, cv2.MORPH_OPEN, np.ones((3,3),np.uint8))

        blue_mask = cv2.morphologyEx(blue_mask, cv2.MORPH_OPEN, np.ones((3,3),np.uint8))
        blue_mask = cv2.morphologyEx(blue_mask, cv2.MORPH_DILATE, np.ones((3,3),np.uint8))

        red = cv2.bitwise_and(self.image, self.image, mask=red_mask)
        blue = cv2.bitwise_and(self.image, self.image, mask=blue_mask)

        red_hsv = cv2.cvtColor(red, cv2.COLOR_BGR2HSV)
        average_hsv_1 = cv2.mean(red_hsv, red_mask1)[:3]
        average_hsv_2 = cv2.mean(red_hsv, red_mask2)[:3] 
        average_hsv_red = cv2.mean(red_hsv,  red_mask)[:3]

        blue_hsv = cv2.cvtColor(blue, cv2.COLOR_BGR2HSV)
        average_hsv_blue = cv2.mean(blue_hsv,  blue_mask)[:3]

        min_value_saturation_red = red_hsv[np.where(red_hsv[:,:,1]>0)][:,1].min() if red_hsv.any() else 100
        min_value_saturation_blue = blue_hsv[np.where(blue_hsv[:,:,1]>0)][:,1].min() if blue_hsv.any() else 100

        red_threshold = (average_hsv_red[1] + min_value_saturation_red) / 2
        blue_threshold = (average_hsv_blue[1] + min_value_saturation_blue) / 2

        max_red1 = 10 if average_hsv_1[0] <= 15 else 20
        min_red2 = 170 if average_hsv_2[0] > 175 else 160

        red_mask1 = cv2.inRange(hsv, (0, red_threshold, 50), (max_red1, 255, 255))
        red_mask2 = cv2.inRange(hsv, (min_red2, red_threshold, 50), (180, 255, 255))
        
        blue_mask = cv2.inRange(hsv, (90, blue_threshold, 50), (130, 255, 255))

        # Merge the mask and crop the red regions
        red_mask = cv2.bitwise_or(red_mask1, red_mask2)
        red_mask = cv2.morphologyEx(red_mask, cv2.MORPH_OPEN, np.ones((3,3),np.uint8))

        blue_mask = cv2.morphologyEx(blue_mask, cv2.MORPH_OPEN, np.ones((3,3),np.uint8))
        blue_mask = cv2.morphologyEx(blue_mask, cv2.MORPH_DILATE, np.ones((3,3),np.uint8))

        red = cv2.bitwise_and(self.image, self.image, mask=red_mask)
        blue = cv2.bitwise_and(self.image, self.image, mask=blue_mask)

        # Show red tracing
        if DEBUG:
            cv2.imshow('Red Color', red)
            cv2.waitKey(0)
            cv2.destroyAllWindows()

        # Show blue tracing
        if DEBUG:
            cv2.imshow('Blue Color', blue)
            cv2.waitKey(0)
            cv2.destroyAllWindows()

        mask = cv2.bitwise_or(red, blue)
        result = cv2.bitwise_and(self.image, mask)

        # Show blue and red colour tracing
        if DEBUG:
            cv2.imshow('Red and Blue Color Detection', result)
            cv2.waitKey(0)
            cv2.destroyAllWindows()
        
        return (red, blue, result)

### Traffic Sign Detection

Function that detects the traffic signs in a given image, drawing their contours and writing their names in that image.

In [120]:
def evaluate_image(image_data):
    color_detector = ColorDetector(image_data.image)
    red_result, blue_result, color_result = color_detector.find_color()

    shape_detector = ShapeDetector(red_result, blue_result, color_result)
    contours = shape_detector.find_shape()

    processed_contours = []
    for t, c, colour in contours:
        if t == "circle":
            center = (c['center'][0], c['center'][1])
        else:
            M = cv2.moments(c)
            cX = int((M["m10"] / (M["m00"] + 1e-7)))
            cY = int((M["m01"] / (M["m00"] + 1e-7)))
            center = (cX, cY)

        contour_radius = c['radius'] if t == "circle" else cv2.minEnclosingCircle(c)[1]
        if not processed_contours:
            processed_contours.append((t, c, colour))
            continue

        invalid_contours = []
        append_contour = False
        processed_verified = 0

        for (tp, cp, colourp) in processed_contours:
            circle_inside = False
            dist = 0
            processed_contour_radius = cp['radius'] if tp == "circle" else cv2.minEnclosingCircle(cp)[1]

            if tp == "circle":
                if (int(cp['center'][0]) - int(center[0]))**2 + (int(cp['center'][1]) - int(center[1]))**2 < processed_contour_radius**2: # Check if center of circle is inside the contour
                    circle_inside = True
            else:
                dist = cv2.pointPolygonTest(cp, center, True)
            if dist > 0 or circle_inside: # Center of the contour is inside a processed contour
                if (contour_radius >= 2/3 * processed_contour_radius and colour == "red" and not (t == "stop" or tp == "stop")) or (contour_radius >= processed_contour_radius and colour != "blue"): # The radius of the contour is bigger than the radius of the processed contour
                    invalid_contours.append((tp, cp, colourp))
                    append_contour = True
            else:
                processed_verified += 1

        for i in invalid_contours:
            processed_contours.remove(i)
        if append_contour or (processed_verified == len(processed_contours)):
            processed_contours.append((t, c, colour))

    signs = []
    stop_count = 0
    rectangle_square = 0
    triangle_count = 0
    
    for t, c, colour in processed_contours:
        signs.append(t)
        if t == "stop": 
            stop_count += 1
        elif t == "square" or t == "rectangle":
            rectangle_square += 1
        elif t == "triangle":
            triangle_count += 1

    dst_found = []

    if stop_count == 0:
        stop_sign = os.path.join("./data/matching", "stop.png")
        dst, kp2, good, sign = matching(stop_sign, image_data.filename, "STOP sign")
        if dst is not None:
            draw_matches(image_data, dst, kp2, good, sign)
            dst_found.append(dst)
    
    if rectangle_square == 0:
        pedestrian1 = os.path.join("./data/matching", "pedestrian_left.png")
        dst_left, kp2_left, good_left, sign_left = matching(pedestrian1, image_data.filename, "blue square")
        pedestrian2 = os.path.join("./data/matching", "pedestrian_right.png")
        dst_right, kp2_right, good_right, sign_right = matching(pedestrian2, image_data.filename, "blue square")

        if dst_left is not None and dst_right is not None:
            if len(good_left) > len(good_right):
                draw_matches(image_data, dst_left, kp2_left, good_left, sign_left)
                dst_found.append(dst_left)
            else:
                draw_matches(image_data, dst_right, kp2_right, good_right, sign_right)
                dst_found.append(dst_right)
        elif dst_left is not None:
            draw_matches(image_data, dst_left, kp2_left, good_left, sign_left)
            dst_found.append(dst_left)
        elif dst_right is not None:
            draw_matches(image_data, dst_right, kp2_right, good_right, sign_right)
            dst_found.append(dst_right)

    if triangle_count == 0:        
        triangle_sign = os.path.join("./data/matching", "bump.png")
        
        dst, kp2, good, sign = matching(triangle_sign, image_data.filename, "red triangle")
        if dst is not None:
            draw_matches(image_data, dst, kp2, good, sign)
            dst_found.append(dst)

    for t, c, colour in processed_contours:
        signs.append(t)
        if t == "circle":
            inside_matching = False
            for dst in dst_found:
                dist = cv2.pointPolygonTest(dst, c['center'], True)
                if dist > 0:
                    inside_matching = True

            if not inside_matching:
                cv2.circle(image_data.image,(c['center']),c['radius'],(0,255,0),2) # Draw the outer circle
                cv2.circle(image_data.image,(c['center']),2,(0,0,255),3) # Draw the center of the circle
                cv2.putText(image_data.image, f"{colour} circle", (c['center'][0] - 35, c['center'][1] - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2) # Write the name of the sign
        else:
            if t == "stop": stop_count += 1
            M = cv2.moments(c)
            cX = int((M["m10"] / (M["m00"] + 1e-7)))
            cY = int((M["m01"] / (M["m00"] + 1e-7)))
            cv2.drawContours(image_data.image, [c], -1, (0, 255, 0), 2)
            classification = f"{colour} {t}" if t != "stop" else "stop sign"
            cv2.putText(image_data.image, classification, (cX - 35, cY - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)
        
    return signs, image_data


Finally, we will run the main code, which processes the requested images.

- Press any key to see the next image, if any
- Press the `Esc` key to exit the application

In [121]:
def process_image(road_number):
    filename = 'road' + str(road_number) + '.png'
    image_data = Data(os.path.join("./data", filename))
    detected_signs, output_image = evaluate_image(image_data)
        
    with open('./data/annotations/road' + str(road_number) + '.xml', 'r') as f: data = f.read()
    root = ET.fromstring(data)

    real_signs = []
    for content in root.findall('.//object/name'):
        real_signs.append(content.text)

    print(f'Detected signs: {detected_signs}')
    print(f'Real signs: {real_signs}')

    # Show the output image
    if OUTPUT == "WINDOW":
        cv2.imshow('Final Result', output_image.image)
    else:
        cv2.imwrite(f'output/{filename}', image_data.image)

if not os.path.exists('./output'):
    os.makedirs('./output')

if FILENAME == "ALL":
    for i in range(52, 877):
        process_image(i)
        k = cv2.waitKey(0)
        cv2.destroyAllWindows()
        if k == 27:
            break
else:
    process_image(FILENAME)

    cv2.waitKey(0)
    cv2.destroyAllWindows()

Detected signs: ['stop', 'stop']
Real signs: ['stop']
Detected signs: ['stop', 'circle', 'circle', 'stop', 'circle', 'circle']
Real signs: ['stop']
Detected signs: ['stop', 'circle', 'stop', 'circle']
Real signs: ['stop']
Detected signs: ['circle', 'circle']
Real signs: ['stop']
Detected signs: ['stop', 'circle', 'circle', 'stop', 'circle', 'circle']
Real signs: ['stop']
Detected signs: ['stop', 'stop']
Real signs: ['stop']
Detected signs: ['stop', 'stop']
Real signs: ['stop']
Detected signs: ['stop', 'stop']
Real signs: ['stop']
Detected signs: ['stop', 'stop']
Real signs: ['stop']
Detected signs: ['stop', 'circle', 'stop', 'circle']
Real signs: ['stop']
Detected signs: ['stop', 'stop']
Real signs: ['stop']
Detected signs: []
Real signs: ['stop']
Detected signs: ['stop', 'stop']
Real signs: ['stop']
Detected signs: ['stop', 'circle', 'circle', 'stop', 'circle', 'circle']
Real signs: ['stop']
Detected signs: ['circle', 'circle', 'circle', 'circle', 'circle', 'circle', 'circle', 'circle